In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from desert import isnum
from pfunctions import getTimeSteps
from FGMTableV2 import *
from readOFFiles import *
from scipy.interpolate import interp1d
from tableProperties import FGMtableProperties
from lookup import lookup
from oneDCaseReader import oneDCaseReader

In [ ]:
FGMtableDict = {}
case1000AdaptiveRoute = "../../Cases/nhep1d/nhep1d1000Adaptive/"
case1000Route         = "../../Cases/nhep1d/nhep1d1000/"
case5000AdaptiveRoute = "../../Cases/nhep1d/nhep1d5000Adaptive/"
case5000Route         = "../../Cases/nhep1d/nhep1d5000/"
case1000DualFuelRoute = "../../Cases/dualFuel1d/DNS1000/"
case4000DualFuelRoute = "../../Cases/dualFuel1d/DNS4000/"


FGMtableDict["route"] = case4000DualFuelRoute
FGMtableDict["Zrange"] = (0.,1.)
FGMtableDict["fuelList"] = ["C7H16","NH3"]
FGMtableDict["gridNumber"] = (301,501)
FGMtableDict["gridPower"] = (2,1)
FGMtableDict["extraLookupFields"] = ["Qdot", "OH","C7H15O2"]
FGMtableDict["PVFields"] = ["N2","CO2","CO","HO2","CH2O","H2O"]
FGMtableDict["PVCoeffs"] = [1.,1.2, 0.9, 2.7, 1.5, 1.2]
FGMtableDict["PVConstant"] = 1.
# FGMtableDict["PVFields"] = ["CO2","CO","HO2","CH2O","H2O"]
# FGMtableDict["PVCoeffs"] = [1.2, 0.9, 2.7, 1.5, 1.2]
# FGMtableDict["PVConstant"] = 0.


obj = FGMtable(FGMtableDict)
obj.Allrun("./","./table_am2a1000/")

In [ ]:
Z0 = obj.Z0
PV0 = obj.PV0
#plt.plot(obj.pdFieldData[0:1000]["PV"])
plt.plot(obj.PVMin)

In [ ]:
obj.getPV0()
Z0 = obj.Z0
PV0 = obj.PV0
plt.plot(Z0,PV0)

In [ ]:
import matplotlib.pyplot as plt

Z = obj.ZList

for index in range(100):
    PV = obj.pdInterpFieldData[index * 301: (index + 1) * 301]["PV"]
    color = (index / 100, index / 100, index / 100)  # Vary shades of gray
    plt.plot(Z, PV, color=color)

plt.xlim([0.03, 0.08])
plt.ylim([0.01, 0.03])
plt.show()

In [ ]:
t = 0.0004
DNSCase = oneDCaseReader("../../Cases/dualFuel1d/DNS1000/")
Zgrid = np.linspace(0,1,301) * np.linspace(0,1,301)
Cgrid = np.linspace(0,1,501)
x_1000 = np.linspace(-0.000512,0.000512,1000)
x_5000 = np.linspace(-0.000512,0.000512,5000)

Z     = DNSCase.readData(t,"Z")
PV    = DNSCase.readData(t,"PV")
C7H16 = DNSCase.readData(t,"C7H16")
T     = DNSCase.readData(t,"T")
RO2   = DNSCase.readData(t,"C7H15O2")
OH    = DNSCase.readData(t,"OH")
HRR   = DNSCase.readData(t,"Qdot")
NH3   = DNSCase.readData(t, "NH3")

c1dLookup = lookup("../tables/c1dtable/",["Z", "scaledPV"], ["T","OH","C7H16","T","C7H15O2","OH","HRR"])

am3Lookup = lookup("./table_am2a4000/",["Z", "scaledPV"], ["T","OH","C7H16","T","C7H15O2","OH","Qdot","NH3"])

import matplotlib.pyplot as plt

plt.figure(figsize = (13,13))
plt.subplot(611)
# Data points
plt.plot(x_1000, T, 'o', markerfacecolor='none', markeredgecolor='black', markersize=6, label="DNS")
plt.plot(x_1000[::3], am3Lookup.lookupList(Z, PV, "T")[0][::3], '-', color='green',markerfacecolor='none', label="method3")

# Customize the plot
plt.xlabel('x / [m]')
plt.ylabel('T / [K]')
plt.xlim([-0.00013, 0.00013])
plt.title('Temperature Distribution')
plt.legend()

plt.subplot(612)
# Data points
plt.plot(x_1000, Z, 'o', markerfacecolor='none', markeredgecolor='black', markersize=6, label="DNS")


# Customize the plot
plt.xlabel('x / [m]')
plt.ylabel(r'$Z$')
plt.xlim([-0.00013, 0.00013])
plt.title('Z Distribution')
plt.legend()

plt.subplot(613)
# Data points
plt.plot(x_1000, NH3, 'o', markerfacecolor='none', markeredgecolor='black', markersize=6, label="DNS")
plt.plot(x_1000[::3], am3Lookup.lookupList(Z, PV, "NH3")[0][::3], '-', color='green',markerfacecolor='none', label="method3")


# Customize the plot
plt.xlabel('x / [m]')
plt.ylabel(r'$Y_{NH3}$')
plt.xlim([-0.00013, 0.00013])
plt.title('NH3 Distribution')
plt.legend()


plt.subplot(614)
# Data points
plt.plot(x_1000, C7H16, 'o', markerfacecolor='none', markeredgecolor='black', markersize=6, label="DNS")
plt.plot(x_1000[::3], am3Lookup.lookupList(Z, PV, "C7H16")[0][::3], '-', color='green',markerfacecolor='none', label="method3")


# Customize the plot
plt.xlabel('x / [m]')
plt.ylabel(r'$Y_{C7H16}$')
plt.xlim([-0.00013, 0.00013])
plt.title('C7H16 Distribution')
plt.legend()


plt.tight_layout()

plt.subplot(615)
# Data points
plt.plot(x_1000, OH, 'o', markerfacecolor='none', markeredgecolor='black', markersize=6, label="DNS")
plt.plot(x_1000[::3], am3Lookup.lookupList(Z, PV, "OH")[0][::3], '-', color='green',markerfacecolor='none', label="method3")

# Customize the plot
plt.xlabel('x / [m]')
plt.ylabel(r'$Y_{OH}$')
plt.xlim([-0.00013, 0.00013])
plt.title('OH Distribution')
plt.legend()

plt.subplot(616)
# Data points
plt.plot(x_1000, RO2, 'o', markerfacecolor='none', markeredgecolor='black', markersize=6, label="DNS")
plt.plot(x_1000[::3], am3Lookup.lookupList(Z, PV, "C7H15O2")[0][::3], '-', color='green',markerfacecolor='none', label="method3")

# Customize the plot
plt.xlabel('x / [m]')
plt.ylabel(r'$Y_{C7H15O2}$')
plt.xlim([-0.00013, 0.00013])
plt.title('C7H15O2 Distribution')
plt.legend()


plt.suptitle("Time = {} ms".format(t*1000), fontsize = 16)
plt.tight_layout()

##plt.savefig("time" + str(t*1000) + ".png",dpi = 100)

In [ ]:
DNSCase = oneDCaseReader("../../Cases/dualFuel1d/DNS1000/")
#c1dCase = oneDCaseReader("../../Cases/nhep1d/c1d/")
m2fCase = oneDCaseReader("../../Cases/dualFuel1d/am2/")
m3fCase = oneDCaseReader("../../Cases/dualFuel1d/am3/")

t = 0.00068
plt.figure(figsize=(12,10))

plt.subplot(511)
x_1000 = np.linspace(-0.000512,0.000512,1000)
field = "T"
field_DNS = DNSCase.readData(t,field)
#field_c1d = c1dCase.readData(t,field)
field_m2f = m2fCase.readData(t,field)
field_m3f = m3fCase.readData(t,field)

plt.plot(x_1000[::5], field_DNS[::5], 'o', markerfacecolor = 'none', markeredgecolor = 'black', label = "DNS")
#plt.plot(x_1000, field_c1d, label = "chem1d")
plt.plot(x_1000, field_m2f, label = "method2 fine")
plt.plot(x_1000, field_m3f, label = "method3 fine")
plt.xlabel("x / [m]")
plt.ylabel("T / [K]")
plt.xlim([-0.00025, 0.00025])
plt.legend()

plt.subplot(512)
x_1000 = np.linspace(-0.000512,0.000512,1000)
field = "NH3"
field_DNS = DNSCase.readData(t,field)
#field_c1d = c1dCase.readData(t,field)
field_m2f = m2fCase.readData(t,field)
field_m3f = m3fCase.readData(t,field)

plt.plot(x_1000[::5], field_DNS[::5], 'o', markerfacecolor = 'none', markeredgecolor = 'black', label = "DNS")
#plt.plot(x_1000, field_c1d, label = "chem1d")
plt.plot(x_1000, field_m2f, label = "method2 fine")
plt.plot(x_1000, field_m3f, label = "method3 fine")
plt.xlabel("x / [m]")
plt.ylabel(r"$Y_{NH3}$")
plt.xlim([-0.00025, 0.00025])
plt.legend()

plt.subplot(513)
x_1000 = np.linspace(-0.000512,0.000512,1000)
field = "C7H16"
field_DNS = DNSCase.readData(t,field)
#field_c1d = c1dCase.readData(t,field)
field_m2f = m2fCase.readData(t,field)
field_m3f = m3fCase.readData(t,field)

plt.plot(x_1000[::5], field_DNS[::5], 'o', markerfacecolor = 'none', markeredgecolor = 'black', label = "DNS")
#plt.plot(x_1000, field_c1d, label = "chem1d")
plt.plot(x_1000, field_m2f, label = "method2 fine")
plt.plot(x_1000, field_m3f, label = "method3 fine")
plt.xlabel("x / [m]")
plt.ylabel(r"$Y_{C7H16}$")
plt.xlim([-0.00025, 0.00025])
plt.legend()

plt.subplot(514)
x_1000 = np.linspace(-0.000512,0.000512,1000)
field = "OH"
field_DNS = DNSCase.readData(t,field)
#field_c1d = c1dCase.readData(t,field)
field_m2f = m2fCase.readData(t,field)
field_m3f = m3fCase.readData(t,field)

plt.plot(x_1000[::5], field_DNS[::5], 'o', markerfacecolor = 'none', markeredgecolor = 'black', label = "DNS")
#plt.plot(x_1000, field_c1d, label = "chem1d")
plt.plot(x_1000, field_m2f, label = "method2 fine")
plt.plot(x_1000, field_m3f, label = "method3 fine")
plt.xlabel("x / [m]")
plt.ylabel(r"$Y_{OH}$")
plt.xlim([-0.00025, 0.00025])
plt.legend()

plt.subplot(515)
x_1000 = np.linspace(-0.000512,0.000512,1000)
field = "C7H15O2"
field_DNS = DNSCase.readData(t,field)
#field_c1d = c1dCase.readData(t,field)
field_m2f = m2fCase.readData(t,field)
field_m3f = m3fCase.readData(t,field)

plt.plot(x_1000[::5], field_DNS[::5], 'o', markerfacecolor = 'none', markeredgecolor = 'black', label = "DNS")
#plt.plot(x_1000, field_c1d, label = "chem1d")
plt.plot(x_1000, field_m2f, label = "method2 fine")
plt.plot(x_1000, field_m3f, label = "method3 fine")
plt.xlabel("x / [m]")
plt.ylabel(r"$Y_{C7H15O2}$")
plt.xlim([-0.00025, 0.00025])
plt.legend()

plt.suptitle("Time = {} ms".format(t*1000),fontsize = 17)
plt.tight_layout()
plt.savefig("timeonline" + str(t*1000) + ".png",dpi = 125)